In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pandas as pd
from itertools import tee

In [2]:
df = pd.read_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SALUD.csv", delimiter=",")

In [75]:
df2

,Unnamed: 0,Unnamed: 0.1,x,y,index_vertice_cercano,LATITUD,LONGITUD,distancia_h,concat,0,...,14,15,16,17,18,14,15,16,17,18
0,0,0,4.124433,-73.627487,33483,4.124,-73.627545,0.048541,4.124%2C-73.6275454,"[742, 4208]",...,None,None,None,None,None,275,653,633,370,759
1,1,1,4.118694,-73.563922,37514,4.119,-73.563647,0.045658,4.119%2C-73.5636471,"[1520, 11132]",...,None,None,None,None,None,1029,1132,1007,1037,1525
2,2,2,4.075202,-73.672954,52077,4.075,-73.672891,0.023477,4.075%2C-73.6728906,"[1325, 10325]",...,None,None,None,None,None,1239,1372,1352,1277,601
3,3,3,4.153048,-73.599674,11110,4.153,-73.599651,0.005913,4.153%2C-73.5996511,"[1173, 5527]",...,None,None,None,None,None,863,997,796,901,1359
4,4,4,4.148361,-73.613292,14637,4.148,-73.613417,0.042406,4.148%2C-73.6134169,"[864, 3861]",...,None,None,None,None,None,750,884,683,788,1171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,195,4.158273,-73.651119,7341,4.158,-73.651163,0.030774,4.158%2C-73.6511635,"[561, 2917]",...,None,None,None,None,None,1145,1327,1308,995,1139
196,196,196,4.103017,-73.659508,45497,4.103,-73.659841,0.036985,4.103%2C-73.6598412,"[950, 6811]",...,None,None,None,None,None,855,988,969,893,217
197,197,197,4.155809,-73.632643,8546,4.156,-73.632766,0.025231,4.156%2C-73.632766,"[408, 1523]",...,None,None,None,None,None,914,1050,1006,765,828
198,198,198,4.136147,-73.609554,23612,4.136,-73.609601,0.017196,4.136%2C-73.609601,"[856, 4328]",...,None,None,None,None,None,489,562,422,543,985


In [4]:
df_nuevos_salud = pd.read_csv("Nuevos_salud.csv", delimiter=";")

In [53]:
df_nuevos_salud

,id,Name,Latitud,Longitud,concat
0,14,Equipamiento potencial 1,4.119692,-73.620595,4.119692117%2C-73.62059494
1,15,Equipamiento potencial 2,4.124112,-73.617205,4.124111665%2C-73.61720463
2,16,Equipamiento potencial 3,4.128911,-73.608692,4.12891072%2C-73.60869169
3,17,Equipamiento potencial 4,4.126492,-73.619860,4.126492293%2C-73.61986041
4,18,Equipamiento potencial 5,4.107019,-73.655174,4.107019258%2C-73.65517375


In [13]:
array_nuevos_salud=df_nuevos_salud["id"].values

In [14]:
array_nuevos_salud

array([14, 15, 16, 17, 18], dtype=int64)

In [39]:
for col_name in array_nuevos_salud:
    df[col_name] = None

In [1]:
#Se quita por motivos de seguridad
key_API_google="" 

In [50]:
df["concat"]=df['LATITUD'].astype(str)+"%2C"+df['LONGITUD'].astype(str)

In [51]:
origenes= np.array(df["concat"].to_list())

In [52]:
df_nuevos_salud["concat"]=df_nuevos_salud['Latitud'].astype(str)+"%2C"+df_nuevos_salud['Longitud'].astype(str)

In [54]:
destinos=np.array(df_nuevos_salud["concat"].to_list())

In [55]:
todas_solicitudes=[]
for i in range(len(origenes)):
    solicitudes=[]
    for j in range(len(destinos)):
        solicitud="https://maps.googleapis.com/maps/api/distancematrix/json?destinations="+destinos[j]+"&origins="+origenes[i]+"&key="+key_API_google
        solicitudes.append(solicitud)
    todas_solicitudes.append(solicitudes)

In [58]:
todas_solicitudes2 = pd.DataFrame(todas_solicitudes)

In [59]:
def solicitud_maps(url):
    try:
        payload = {}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        x = response.json()
        drive_time_seconds = x['rows'][0]['elements'][0]['duration']['value']
        distance_meters = x['rows'][0]['elements'][0]['distance']['value']
        return [drive_time_seconds, distance_meters]
    except KeyError:
        return 0

In [60]:
df["14"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[0]), axis=1)

In [62]:
df["15"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[1]), axis=1)
df["16"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[2]), axis=1)
df["17"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[3]), axis=1)
df["18"] = todas_solicitudes2.apply(lambda x: solicitud_maps(x[4]), axis=1)

In [64]:
df.to_csv("FINAL_tiempos_distancia_saludnuevos.csv",index=True)

In [65]:
df2=df.copy(deep=True)

In [67]:
array_nuevos_salud

array([14, 15, 16, 17, 18], dtype=int64)

In [71]:
df2["15"] = df2["15"].apply(lambda x: x[0])

In [74]:
df2["14"] = df2["14"] .apply(lambda x: x[0])
df2["16"] = df2["16"] .apply(lambda x: x[0])
df2["17"] = df2["17"] .apply(lambda x: x[0])
df2["18"] = df2["18"] .apply(lambda x: x[0])

In [76]:
df2.to_csv("TIEMPOS_VIAJE_SEG_CLUSTERS_SALUDNUEVOS.csv",index=True)